In [9]:
import pandas as pd

from open_quant_data.dataset.thirdparty.akshare.AkshareDataset import AkshareDataset as ak

In [23]:
sh_names = ak.sh_stock_names()
# sh_names.to_excel('sh_names.xlsx')
sh_names

,证券代码,证券简称,公司全称,上市日期
0,600000,浦发银行,上海浦东发展银行股份有限公司,1999-11-10
1,600004,白云机场,广州白云国际机场股份有限公司,2003-04-28
2,600006,东风汽车,东风汽车股份有限公司,1999-07-27
3,600007,中国国贸,中国国际贸易中心股份有限公司,1999-03-12
4,600008,首创环保,北京首创生态环保集团股份有限公司,2000-04-27
...,...,...,...,...
1682,605580,恒盛能源,恒盛能源股份有限公司,2021-08-19
1683,605588,冠石科技,南京冠石科技股份有限公司,2021-08-12
1684,605589,圣泉集团,济南圣泉集团股份有限公司,2021-08-10
1685,605598,上海港湾,上海港湾基础建设(集团)股份有限公司,2021-09-17


In [24]:
sz_names = ak.sz_stock_names()
sz_names

,板块,A股代码,A股简称,A股上市日期,A股总股本,A股流通股本,所属行业
0,主板,000001,平安银行,1991-04-03,"19,405,918,198","19,405,546,950",J 金融业
1,主板,000002,万 科Ａ,1991-01-29,"9,724,196,533","9,716,577,665",K 房地产
2,主板,000004,国华网安,1990-12-01,"132,380,282","126,288,093",I 信息技术
3,主板,000005,ST星源,1990-12-10,"1,058,536,842","1,057,875,742",N 公共环保
4,主板,000006,深振业Ａ,1992-04-27,"1,349,995,046","1,349,987,396",K 房地产
...,...,...,...,...,...,...,...
2824,创业板,301550,斯菱股份,2023-09-15,"110,000,000","26,081,889",C 制造业
2825,创业板,301555,惠柏新材,2023-10-31,"92,266,700","21,876,227",C 制造业
2826,创业板,301558,三态股份,2023-09-28,"788,851,223","111,231,855",F 批发零售
2827,创业板,301559,中集环科,2023-10-11,"600,000,000","84,506,418",C 制造业


In [25]:
ak.a_stock_names()

,stock_code,stock_name,begin_date
0,600000,浦发银行,1999-11-10
1,600004,白云机场,2003-04-28
2,600006,东风汽车,1999-07-27
3,600007,中国国贸,1999-03-12
4,600008,首创环保,2000-04-27
...,...,...,...
2824,301550,斯菱股份,2023-09-15
2825,301555,惠柏新材,2023-10-31
2826,301558,三态股份,2023-09-28
2827,301559,中集环科,2023-10-11


In [31]:
from datetime import datetime, date

dataset = ak.stock_basic_finance_index('000001')
dataset.rename(columns={'trade_date': 'date'}, inplace=True)
dataset = dataset[dataset['date'] >= date(2022, 1, 1)]
dataset.to_excel('000001-2022-fin.xlsx')

In [32]:
dataset = ak.stock_date('sh000001')
dataset = dataset[dataset['date'] >= datetime(2010, 1, 1)]
dataset.to_excel('000001-2022-price.xlsx')

In [10]:
from datetime import datetime, date
import pandas as pd


def stock_price_finance(stock_id: str, start_date: datetime) -> pd.DataFrame:
    # transform prefix stock id
    prefix_stock_id = ''
    if stock_id.startswith('6'):
        prefix_stock_id = 'sh' + stock_id
    elif stock_id.startswith('0') or stock_id.startswith('3'):
        prefix_stock_id = 'sz' + stock_id
    price_dataset = ak.stock_date(prefix_stock_id)
    finance_dataset = ak.stock_basic_finance_index(stock_id)
    # transform timestamp
    start_timestamp = start_date.strftime('%Y-%m-%d')
    fin_start_date = finance_dataset['trade_date'].tolist()[0]
    price_start_date = price_dataset['date'].tolist()[0].date()
    result_start_date = start_date.date()
    if result_start_date < fin_start_date:
        result_start_date = fin_start_date
    if result_start_date < price_start_date:
        result_start_date = price_start_date
    # pre-handle
    finance_dataset.rename(columns={'trade_date': 'date'}, inplace=True)
    finance_dataset = finance_dataset[finance_dataset['date'] >= result_start_date]
    finance_dataset = finance_dataset.fillna(value=0)
    price_dataset = price_dataset[
        price_dataset['date'] >= datetime(result_start_date.year, result_start_date.month, result_start_date.day)]
    price_dataset = price_dataset.fillna(value=0)
    # transform index
    finance_dataset['date'] = pd.to_datetime(finance_dataset['date'], format="%Y-%m-%d")
    price_dataset['date'] = pd.to_datetime(price_dataset['date'], format="%Y-%m-%d")
    result_dataset = pd.merge(price_dataset, finance_dataset, how='inner', on='date')

    result_dataset['ret'] = result_dataset['close'].pct_change(1)
    result_dataset.insert(loc=2, column='stock_id', value=stock_id)
    result_dataset = result_dataset.fillna(value=0)
    return result_dataset

# result_dataset = stock_price_finance('600000', datetime(2010, 1, 1))
# result_dataset.to_excel('600000-total.xlsx')
# result_dataset

In [4]:
names = ak.a_stock_names()
cnt = 4515
length = len(names['stock_code'].tolist())
for i in range(4514, length):
    code = names['stock_code'].tolist()[i]
    result_dataset = stock_price_finance(code, datetime(2010, 1, 1))
    result_dataset.to_excel(f'./output/{code}.xlsx')
    print(f'finish {code}, {cnt} / {length}')
    cnt += 1

finish 301559, 4515 / 4516


ValueError: 数据获取失败, 请检查是否输入正确的股票代码

In [15]:
def latest_stock_price_finance() -> pd.DataFrame:
    names = ak.a_stock_names()
    cnt = 0
    length = len(names['stock_code'].tolist())
    combined_df = pd.DataFrame()
    for i in range(0, length):
        code = names['stock_code'].tolist()[i]
        result_dataset = stock_price_finance(code, datetime(2023, 11, 30))
        combined_df = pd.concat([combined_df, result_dataset.iloc[-1:]])
        print(f'finish {code}, {cnt} / {length}')
        cnt += 1
    return combined_df

dataset = latest_stock_price_finance()
dataset

finish 600000, 0 / 4516
finish 600004, 1 / 4516
finish 600006, 2 / 4516
finish 600007, 3 / 4516
finish 600008, 4 / 4516
finish 600009, 5 / 4516
finish 600010, 6 / 4516
finish 600011, 7 / 4516
finish 600012, 8 / 4516
finish 600015, 9 / 4516
finish 600016, 10 / 4516
finish 600017, 11 / 4516
finish 600018, 12 / 4516
finish 600019, 13 / 4516
finish 600020, 14 / 4516
finish 600021, 15 / 4516
finish 600022, 16 / 4516
finish 600023, 17 / 4516
finish 600025, 18 / 4516
finish 600026, 19 / 4516
finish 600027, 20 / 4516
finish 600028, 21 / 4516
finish 600029, 22 / 4516
finish 600030, 23 / 4516
finish 600031, 24 / 4516
finish 600032, 25 / 4516
finish 600033, 26 / 4516
finish 600035, 27 / 4516
finish 600036, 28 / 4516
finish 600037, 29 / 4516
finish 600038, 30 / 4516
finish 600039, 31 / 4516
finish 600048, 32 / 4516
finish 600050, 33 / 4516
finish 600051, 34 / 4516
finish 600052, 35 / 4516
finish 600053, 36 / 4516
finish 600054, 37 / 4516
finish 600055, 38 / 4516
finish 600056, 39 / 4516
finish 600


KeyboardInterrupt

